In [6]:
#%pip install wbdata
import wbdata
import numpy as np
import pandas as pd

ModuleNotFoundError: No module named 'wbdata'

# 1. Population DataFrames [A]

In [ ]:

def population_df(years, age_range, area) -> pd.DataFrame: 
    """
    years (tuple): (start_year, end_year), use (year, year) if just want singular year
    age_range (tuple): (low_age, high_age)
    area (str): e.g. "India"

    This function takes in year, age_range, and area [country/world] and outputs a DataFrame with relevant information
    """
    country_id = wbdata.get_countries(query=area)[0]["id"]

    low, high = age_range
    ages = range(low, high + 1)

    indicators = {}
    for age in ages:
        age2 = f"{age:02d}"
        indicators[f"SP.POP.AG{age2}.MA.IN"] = f"male_{age2}"
        indicators[f"SP.POP.AG{age2}.FE.IN"] = f"female_{age2}"

    df = wbdata.get_dataframe(indicators, country=country_id, parse_dates=True)

    start, end = years
    df = df[(df.index.year >= start) & (df.index.year <= end)]

    return df

### Population Df of India from 2010, male and female age 1-3

In [ ]:
india_df = population_df((2000, 2010), (1, 3), "India")
india_df

# 2. Population Statistics [A]

In [ ]:
def population(years, sex, age_range, area):
    """
    years (tuple): (start_year, end_year) ,use (year, year) if just want singular year
    sex (str): "Male", "Female", or "All"
    age_range (tuple): (low_age, high_age)
    area (str): e.g. "India", "World"

    This function answers the question: 
    In [year] how many [people/males/females] aged [low] to [high] were living in [the world/region/country]?
    The output is a Series
    """
    df = population_df(years, age_range, area)
    df['male_total'] = df.filter(like='male_').sum(axis=1)
    df['female_total'] = df.filter(like='female_').sum(axis=1)
    df['population_total'] = df['female_total']+ df['male_total']
    
    #only the aggregated df
    agg = df[['male_total', 'female_total', 'population_total']]
    agg = agg.copy()
    agg.index = agg.index.year
    agg.index.name = "year"
    
    if sex == 'Female':
        return agg['female_total']
    elif sex == 'Male':
        return agg['male_total']
    elif sex == 'All':
        return agg['population_total']
    else:
        print("Invalid Input. Please input 'Female', 'Male', or 'All'.")
        
        



In [ ]:
india_pop = population((1960, 1980), 'Female', (0, 3), 'India')
india_pop.plot()

In [ ]:
#population plots
female = {"SP.POP.TOTL.FE.IN" : "Population, female"}
male = {"SP.POP.TOTL.MA.IN" : "Population, male"}

#creating dataframe of populations by country, organized by gender
def get_genders_df(country, gender):
    
    def country_id(country):
        single_country = wbdata.get_countries(query = country)
        return single_country[0]["id"]
        
    country_id = country_id(country) 

    female_df = wbdata.get_dataframe(female, country=country_id,parse_dates=True)
    male_df = wbdata.get_dataframe(male, country=country_id,parse_dates=True)
    if gender == "female": 
        return female_df 
    if gender == "male":
        return male_df 
    if gender == "female and male":
        return female_df.join(male_df)
    else: 
        return "Format gender as 'male', 'female', or 'female and male'"

def population_trend_on_event(country, gender, date):
    gender_df = get_genders_df(country, gender)
    return gender_df.where("date", are.containing(date))
    
    
def plot_df(country):
    df = get_gender_df(country)
    plotted = df.plot()
    return plotted.set_title( country + "'s population by gender (1960-Present)")
    
#get_genders_df("India", "male")

df_fe = {"SH.DYN.MORT.FE" : "Mortality rate, under-5, female (per 1,000 live births)"}
#def mortality(years, 
india_fe_mortality = wbdata.get_dataframe(df_fe, country = "IND", parse_dates = True)
#start, end = years
   # df = df[(df.index.year >= start) & (df.index.year <= end)]


india_fe_mortality.plot()


In [ ]:
india_fe_mortality 


NameError: name 'wbdata' is not defined

In [42]:
type(india_fe_mortality.index)

pandas.core.indexes.datetimes.DatetimeIndex